In [7]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Cassandra
from dotenv import load_dotenv
import cassio

In [8]:
load_dotenv()

True

## Getting API keys and token from environment

In [10]:
groq_api_key = os.getenv("GROQ_API_KEY")
ASTRA_DB_TOKEN = os.getenv("ASTRADBTOKEN")
ASTRA_DB_ID = os.getenv("ASTRADBID")


## Connecting to the AstraDB

In [11]:
cassio.init(token=ASTRA_DB_TOKEN, database_id=ASTRA_DB_ID)